In [37]:
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
import torch
# from sentence_transformers import SentenceTransformer
# from spellchecker import SpellChecker
import string
import matplotlib.pyplot as plt
from collections import Counter


# Apply tqdm to all .apply() functions by using progress_apply
tqdm.pandas()

In [38]:
# Use GPU if available
"""
Reference: https://pytorch.org/get-started/locally/
"""

# Check for NVIDIA GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA (NVIDIA GPU)
    print("Using NVIDIA GPU (CUDA)")

# Check for Mac Silicon GPU (MPS)
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal Performance Shaders (Mac Silicon GPU)
    print("Using Mac GPU (MPS)")

# Default to CPU if no GPU is available
else:
    device = torch.device("cpu")
    print("Using CPU")

Using NVIDIA GPU (CUDA)


In [39]:
# spell = SpellChecker()

In [40]:
# Reading the dataset
alexator_df = pd.read_csv('../data/alexator_stickers_desc.csv')
# flaticon_df = pd.read_csv('../data/flaticon_stickers_desc.csv')
# freepik_df = pd.read_csv('../data/freepik_stickers_desc.csv')

In [41]:
alexator_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     4331 non-null   object
 1   description  4331 non-null   object
dtypes: object(2)
memory usage: 67.8+ KB


In [42]:
# flaticon_df.info()

In [43]:
# freepik_df.info()

In [44]:
# Concatenate the DataFrames row-wise
# merged_df = pd.concat([alexator_df, flaticon_df, freepik_df], ignore_index=True)
merged_df = alexator_df

# Display the concatenated DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     4331 non-null   object
 1   description  4331 non-null   object
dtypes: object(2)
memory usage: 67.8+ KB


## Handling duplicates in description and tags combination


In [45]:
# # Replace duplicate descriptions with Null (Because most of them don't make sense and are incorrect)
# duplicate_descriptions = merged_df['description'].duplicated(keep=False)
# merged_df.loc[duplicate_descriptions, 'description'] = None
# 
# """Some generated descriptions have errors such as "a big boy boy boy boy boy...."
# So, we want to remove such descriptions that have incorrect descriptions
# """
# 
# """
# Reference: ChatGPT-4o
# Prompt: I need to check for a certain type of error and remove those rows in my dataframe. The error is that in some description, certain words are repeated multiple time consecutively. Example: "happy man with two polar polar polar polar polar polar polar polar polar polar polar polar polar polar polar polar polar"
# """
# # Replace descriptions with repeated words with Null
# def has_repeated_words(text):
#     if not text or pd.isna(text):  # Skip null or empty strings
#         return False
#     
#     # Regex to find words repeated consecutively at least 3 times (case insensitive)
#     pattern = r'\b(\w+)(?:\s+\1){2,}\b'
#     return bool(re.search(pattern, text, re.IGNORECASE))
# 
# merged_df.loc[merged_df['description'].apply(has_repeated_words), 'description'] = None
# 
# # Remove rows with exact same tags AND description, keeping the first occurrence (We don't want different immages with same details)
# cleaned_df = merged_df.drop_duplicates(subset=['tags', 'description'], keep='first')
# 
# # Check the new number of rows
# print(f"Number of rows after removing rows with exact same tags and descriptions: {cleaned_df.shape[0]}")

In [46]:
# cleaned_df.info()

## Handling Tags and Descriptions

In [47]:
# def remove_tags_duplicates(text):
#     if not isinstance(text, str) or pd.isna(text) or text.strip().lower() == "nan":  
#         return ""  # Return empty string for NaN or "nan" strings
#     unique_words = sorted(set(word.strip() for word in text.split(",")))
#     return ', '.join(unique_words)  # Join back into a string

In [48]:
# Remove duplicates
# final_df = cleaned_df.copy()
# final_df["tags"] = final_df["tags"].progress_apply(remove_tags_duplicates)

In [49]:
### Need to conduct spelling correction in description column here as well

In [50]:
# # Function to handle concatenation with empty strings and NaN values
# def merge_descriptions(row):
#     parts = []
#     if pd.notna(row['description']) and row['description'].strip():
#         parts.append(row['description'].strip())
#     if pd.notna(row['tags']) and row['tags'].strip():
#         parts.append("Tags: " + row['tags'].strip())
#     
#     return " ".join(parts)

final_df = merged_df.copy()
# Apply the function to each row

In [51]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   filename     4331 non-null   object
 1   description  4331 non-null   object
dtypes: object(2)
memory usage: 67.8+ KB


In [52]:
# def check_misspelled_words(text):
#     if not isinstance(text, str) or text.strip() == "":
#         return []
# 
#     translator = str.maketrans('', '', string.punctuation)
#     clean_text = text.translate(translator).lower()
#     words = clean_text.split()
# 
#     misspelled = list(spell.unknown(words))
#     return misspelled

In [53]:
# final_df['misspelled_words'] = final_df['merged_description'].progress_apply(check_misspelled_words)

In [54]:
# # Filter rows where spelling_errors is not an empty list
# rows_with_errors = final_df[final_df['misspelled_words'].apply(lambda x: bool(x))]
# 
# rows_with_errors["misspelled_words"]

In [55]:
# count = final_df['spelling_errors'].apply(lambda x: bool(x)).sum()
# print(f"Number of rows with spelling errors: {count}")

In [56]:
# from textblob import TextBlob
# 
# def textblob_correct(text):
#     if not isinstance(text, str) or text.strip() == "":
#         return ""
#     try:
#         blob = TextBlob(text)
#         return str(blob.correct())
#     except Exception as e:
#         print(f"Failed to correct: {text[:50]}... — {e}")
#         return text

In [57]:
# final_df['corrected_description'] = final_df['merged_description'].progress_apply(textblob_correct)

In [58]:
# # Filter rows where spelling_errors is not an empty list
# final_df[final_df['misspelled_words'].apply(lambda x: bool(x))][["corrected_description", "misspelled_words"]]

## Exploratory Business Analysis

In [59]:
# # Text Length Distribution
# final_df["desc_length"] = final_df["description"].apply(lambda x: len(str(x).split()))
# 
# plt.hist(final_df["desc_length"], bins=30)
# plt.title("Distribution of Description Word Counts")
# plt.xlabel("Number of Words")
# plt.ylabel("Frequency")
# plt.show()


In [60]:
# # Tag Frequency Analysis
# tag_list = final_df['tags'].dropna().apply(lambda x: [tag.strip() for tag in x.split(",")])
# all_tags = [tag for sublist in tag_list for tag in sublist]
# 
# top_tags = Counter(all_tags).most_common(20)
# 
# print("Top 20 tags:")
# print(top_tags)

In [61]:
# tag_df = pd.DataFrame(top_tags, columns=["tag", "count"])
# tag_df.plot(kind='bar', x='tag', y='count', figsize=(10,5), title="Top 20 Tags")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

In [62]:
# from nltk.corpus import stopwords
# import nltk
# 
# # Most Common Words
# nltk.download('stopwords')
# 
# stop_words = set(stopwords.words('english'))
# 
# word_list = final_df['description'].dropna().apply(lambda x: [w for w in str(x).lower().split() if w not in stop_words])
# flat_words = [word for sublist in word_list for word in sublist]
# 
# top_words = Counter(flat_words).most_common(20)
# print("Top 20 common words:")
# print(top_words)


## Embedding using Sentence BERT

In [63]:
# # Load SBERT model
# sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
# sbert_model = sbert_model.to(device)
# 
# # Return SBERT embedding for a given text.
# def embed_text(text):
#     if pd.isna(text) or text.strip() == "":
#         return np.zeros(384, dtype=np.float32)  # Return zero vector for missing values (SBERT output size = 384)
#     return sbert_model.encode(text).astype(np.float32)

In [64]:
"""
Reference: https://sbert.net/
"""

# Apply SBERT embeddings to each columns
# print("Embedding tags...")
# final_df["tags_embedding"] = final_df["tags"].progress_apply(embed_text)
# 
# print("Embedding llm_description...")
# final_df["description_embedding"] = final_df["description"].progress_apply(embed_text)
# print("Embedding text...")
# final_df["combined_embedding"] = final_df["merged_description"].progress_apply(embed_text)

'\nReference: https://sbert.net/\n'

In [65]:
# # Combining the embeddings
# final_df["combined_embedding"] = final_df.apply(
#     lambda row: np.concatenate([row["tags_embedding"], row["description_embedding"]]), axis=1
# )

## Linking Images

In [66]:
final_df.head()

,filename,description
0,cartoonized_image_0.png,a red mailbox with a white background
1,cartoonized_image_1.png,an old car sticker
2,cartoonized_image_10.png,hot air balloon sticker
3,cartoonized_image_100.png,a sticker with a cartoon pine wearing glasses
4,cartoonized_image_1000.png,a stained stained glass window with a humming


In [67]:
# Change file extensions from .png to .pt in the 'filename' column
final_df['filename'] = final_df['filename'].str.replace('.png', '.pt', regex=False)

# Display the first few rows to verify changes
final_df['filename'].head()

0       cartoonized_image_0.pt
1       cartoonized_image_1.pt
2      cartoonized_image_10.pt
3     cartoonized_image_100.pt
4    cartoonized_image_1000.pt
Name: filename, dtype: object

In [68]:
# Define the base directory and folders to search in
base_dir = "../data/tensor_images/"
folders = ["AlexatorStickers"]

# Function to find the file path for a given filename
def find_image_path(filename):
    # Loop through each folder
    for folder in folders:
        # Construct full path
        full_path = os.path.join(base_dir, folder, filename)
        # Check if file exists at this path
        if os.path.exists(full_path):
            return full_path
    # Return None if file not found in any folder
    return None

# Create the new column by applying the function to the filename column
final_df['image_path'] = final_df['filename'].progress_apply(find_image_path)

# Display a sample of the DataFrame to verify
final_df.head()

100%|██████████| 4331/4331 [00:00<00:00, 62338.17it/s]


,filename,description,image_path
0,cartoonized_image_0.pt,a red mailbox with a white background,../data/tensor_images/AlexatorStickers\cartoon...
1,cartoonized_image_1.pt,an old car sticker,../data/tensor_images/AlexatorStickers\cartoon...
2,cartoonized_image_10.pt,hot air balloon sticker,../data/tensor_images/AlexatorStickers\cartoon...
3,cartoonized_image_100.pt,a sticker with a cartoon pine wearing glasses,../data/tensor_images/AlexatorStickers\cartoon...
4,cartoonized_image_1000.pt,a stained stained glass window with a humming,../data/tensor_images/AlexatorStickers\cartoon...


In [69]:
final_df.rename(columns={'description': 'prompt'}, inplace=True)


In [70]:
final_df.tail()

,filename,prompt,image_path
4326,cartoonized_image_995.pt,a cartoon fox sitting in a field of flowers,../data/tensor_images/AlexatorStickers\cartoon...
4327,cartoonized_image_996.pt,a cartoon frog sitting on a table,../data/tensor_images/AlexatorStickers\cartoon...
4328,cartoonized_image_997.pt,a kangaroo playing the piano,../data/tensor_images/AlexatorStickers\cartoon...
4329,cartoonized_image_998.pt,a sticker with a landscape and a butterfly,../data/tensor_images/AlexatorStickers\cartoon...
4330,cartoonized_image_999.pt,flaming flaming flaming flaming flaming flamin...,../data/tensor_images/AlexatorStickers\cartoon...


In [71]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4331 entries, 0 to 4330
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filename    4331 non-null   object
 1   prompt      4331 non-null   object
 2   image_path  4331 non-null   object
dtypes: object(3)
memory usage: 101.6+ KB


In [72]:
final_df.to_parquet('../data/processed_sticker_dataset.parquet', index=False)